<a href="https://colab.research.google.com/github/vothane/II4I/blob/master/II4I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Lets go Bruins. All eyes on center.**

![](https://https://media1.giphy.com/media/hJjTwuJjjsVgc/giphy.gif)

![](https://media1.giphy.com/media/hJjTwuJjjsVgc/giphy.gif)

In [0]:
import os
import tensorflow as tf
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

In [0]:
## Functions for Capsule Networks
def squash(vectors, name=None):
    """
    Squashing Function
    :parameter vectors: vector input that needs to be squashed
    :parameter name: Name of the tensor on the graph
    :return: a tensor with same shape as vectors but squashed as mentioned in the paper
    """
    with tf.name_scope(name, default_name="squash_op"):
        s_squared_norm = tf.reduce_sum(tf.square(vectors), axis=-2, keepdims=True)
        scale = s_squared_norm / (1. + s_squared_norm) / tf.sqrt(s_squared_norm + tf.keras.backend.epsilon())
        return scale*vectors


def routing(u):
    """
    This function performs the routing algorithm
    :parameter u: Input tensor with [batch_size, num_caps_input_layer=1152, 1, caps_dim_input_layer=8, 1] shape.
                NCAPS_CAPS1: num capsules in the PrimaryCaps layer l
                CAPS_DIM_CAPS2: dimensions of output vectors of Primary caps layer l
    :return: "v_j" vector (tensor) in Digitcaps Layer
             Shape:[batch_size, NCAPS_CAPS1=10, CAPS_DIM_CAPS2=16, 1]
    """

    #local variable b_ij: [batch_size, num_caps_input_layer=1152, num_caps_output_layer=10, 1, 1]
                #num_caps_output_layer: number of capsules in Digicaps layer l+1
    b_ij = tf.zeros([BATCH_SIZE, NCAPS_CAPS1, NCAPS_CAPS2, 1, 1], dtype=np.float32, name="b_ij")

    # Preparing the input Tensor for total number of DigitCaps capsule for multiplication with W
    u = tf.tile(u, [1, 1, b_ij.shape[2].value, 1, 1])   # u => [batch_size, 1152, 10, 8, 1]


    # W: [num_caps_input_layer, num_caps_output_layer, len_u_i, len_v_j] as mentioned in the paper
    W = tf.get_variable('W', shape=(1, u.shape[1].value, b_ij.shape[2].value, u.shape[3].value, CAPS_DIM_CAPS2),
                        dtype=tf.float32, initializer=tf.random_normal_initializer(stddev=STDEV))
    W = tf.tile(W, [BATCH_SIZE, 1, 1, 1, 1]) # W => [batch_size, 1152, 10, 8, 16]

    #Computing u_hat (as mentioned in the paper)
    u_hat = tf.matmul(W, u, transpose_a=True)  # [batch_size, 1152, 10, 16, 1]

    # In forward, u_hat_stopped = u_hat;
    # In backward pass, no gradient pass from  u_hat_stopped to u_hat
    u_hat_stopped = tf.stop_gradient(u_hat, name='gradient_stop')

    # Routing Algorithm Begins here
    for r in range(ROUTING_ITERATIONS):
        with tf.variable_scope('iterations_' + str(r)):
            c_ij = tf.nn.softmax(b_ij, axis=2) # [batch_size, 1152, 10, 1, 1]

            # At last iteration, use `u_hat` in order to back propagate gradient
            if r == ROUTING_ITERATIONS - 1:
                s_j = tf.multiply(c_ij, u_hat) # [batch_size, 1152, 10, 16, 1]
                # then sum as per paper
                s_j = tf.reduce_sum(s_j, axis=1, keep_dims=True) # [batch_size, 1, 10, 16, 1]

                v_j = squash(s_j) # [batch_size, 1, 10, 16, 1]

            elif r < ROUTING_ITERATIONS - 1:  # No backpropagation in these iterations
                s_j = tf.multiply(c_ij, u_hat_stopped)
                s_j = tf.reduce_sum(s_j, axis=1, keepdims=True)
                v_j = squash(s_j)
                v_j = tf.tile(v_j, [1, u.shape[1].value, 1, 1, 1]) # [batch_size, 1152, 10, 16, 1]

                # Multiplying in last two dimensions: [16, 1]^T x [16, 1] yields [1, 1]
                u_hat_dot_v = tf.matmul(u_hat_stopped, v_j, transpose_a=True) # [batch_size, 1152, 10, 1, 1]

                b_ij = tf.add(b_ij,u_hat_dot_v)
    return tf.squeeze(v_j, axis=1) # [batch_size, 10, 16, 1]

  
def shuffle_data(x, y):
    """ Shuffle the features and labels of input data"""
    perm = np.arange(y.shape[0])
    np.random.shuffle(perm)
    shuffle_x = x[perm,:,:,:]
    shuffle_y = y[perm]
    return shuffle_x, shuffle_y


def load_existing_details():
    """
    This function loads the train and val files to continue training
    :return: handles to train and val files and minimum validation loss
    """
    train_path = RESULTS_DIR  + '/' + 'train.csv'
    val_path = RESULTS_DIR + '/' + 'validation.csv'
    # finding the minimum validation loss so far
    f_val = open(val_path, 'r')
    lines = f_val.readlines()
    data = np.genfromtxt(lines[-1:], delimiter=',')
    min_loss = np.min(data[1:, 2])
    # loading the train and val files to continue training
    train_file = open(train_path, 'a')
    val_file = open(val_path, 'a')
    return train_file, val_file, min_loss


def eval_performance(sess, model, x, y):
    '''
    This function is mainly used to evaluate the accuracy on test and validation sets
    :param sess: session
    :param model: model to be used
    :param x: images
    :param y: labels
    :return: returns the average accuracy and loss for the dataset
    '''
    acc_all = loss_all = np.array([])
    num_batches = int(y.shape[0] / BATCH_SIZE)
    for batch_num in range(num_batches):
        start = batch_num * BATCH_SIZE
        end = start + BATCH_SIZE
        x_batch, y_batch = x[start:end], y[start:end]
        acc_batch, loss_batch, prediction_batch = sess.run([model.accuracy, model.combined_loss, model.y_predicted],
                                                     feed_dict={model.X: x_batch, model.Y: y_batch})
        acc_all = np.append(acc_all, acc_batch)
        loss_all = np.append(loss_all, loss_batch)
    return np.mean(acc_all), np.mean(loss_all)

def reconstruction(x, y, decoder_output, y_pred, n_samples):
    '''
    This function is used to reconstruct sample images for analysis
    :param x: Images
    :param y: Labels
    :param decoder_output: output from decoder
    :param y_pred: predictions from the model
    :param n_samples: num images
    :return: saves the reconstructed images
    '''

    sample_images = x.reshape(-1, IMG_WIDTH, IMG_HEIGHT)
    decoded_image = decoder_output.reshape([-1, IMG_WIDTH, IMG_WIDTH])

    fig = plt.figure(figsize=(n_samples * 2, 3))
    for i in range(n_samples):
        plt.subplot(1, n_samples, i+ 1)
        plt.imshow(sample_images[i], cmap="binary")
        plt.title("Label:" + IMAGE_LABELS[np.argmax(y[i])])
        plt.axis("off")
    fig.savefig(RESULTS_DIR + '/' + 'input_images.png')
    plt.show()

    fig = plt.figure(figsize=(n_samples * 2, 3))
    for i in range(n_samples):
        plt.subplot(1, n_samples, i + 1)
        plt.imshow(decoded_image[i], cmap="binary")
        plt.title("Prediction:" + IMAGE_LABELS[y_pred[i]])
        plt.axis("off")
    fig.savefig(RESULTS_DIR + '/' + 'decoder_images.png')
    plt.show()


In [0]:
class CapsNet:
    def __init__(self):
        with tf.variable_scope('Input'):
            self.X = tf.placeholder(shape=[None, IMG_WIDTH, IMG_HEIGHT, N_CHANNELS], dtype=tf.float32, name="X")
            self.Y = tf.placeholder(shape=[None, N_CLASSES], dtype=tf.float32, name="Y")
            self.mask_with_labels = tf.placeholder_with_default(False, shape=(), name="mask_with_labels")

        self.define_network()
        self.define_loss()
        self.define_accuracy()
        self.define_optimizer()
        self.summary_()

    def define_network(self):
        with tf.variable_scope('Conv1_layer'):
            conv1_layer = tf.layers.conv2d(self.X, name="conv1_layer", **CONV1_LAYER_PARAMS) # [batch_size, 20, 20, 256]

        with tf.variable_scope('PrimaryCaps_layer'):
            conv2_layer = tf.layers.conv2d(conv1_layer, name="conv2_layer", **CONV2_LAYER_PARAMS) # [batch_size, 6, 6, 256]

            primary_caps = tf.reshape(conv2_layer, (BATCH_SIZE, NCAPS_CAPS1, CAPS_DIM_CAPS1, 1), name="primary_caps") # [batch_size, 1152, 8, 1]
            primary_caps_output = squash(primary_caps, name="caps1_output")
            # [batch_size, 1152, 8, 1]

        # DigitCaps layer, return [batch_size, 10, 16, 1]
        with tf.variable_scope('DigitCaps_layer'):
            digitcaps_input = tf.reshape(primary_caps_output, shape=(BATCH_SIZE, NCAPS_CAPS1, 1, CAPS_DIM_CAPS1, 1)) # [batch_size, 1152, 1, 8, 1]
            # [batch_size, 1152, 10, 1, 1]
            self.digitcaps_output = routing(digitcaps_input) # [batch_size, 10, 16, 1]

        # Decoder
        with tf.variable_scope('Masking'):
            self.v_norm = tf.sqrt(tf.reduce_sum(tf.square(self.digitcaps_output), axis=2, keep_dims=True) + tf.keras.backend.epsilon())

            predicted_class = tf.to_int32(tf.argmax(self.v_norm, axis=1)) #[batch_size, 10,1,1]
            self.y_predicted = tf.reshape(predicted_class, shape=(BATCH_SIZE,))  #[batch_size]
            y_predicted_one_hot = tf.one_hot(self.y_predicted, depth=NCAPS_CAPS2)  #[batch_size,10]  One hot operation

            reconstruction_targets = tf.cond(self.mask_with_labels,  # condition
                                      lambda: self.Y,  # if True (Training)
                                      lambda: y_predicted_one_hot,  # if False (Test)
                                      name="reconstruction_targets")

            digitcaps_output_masked = tf.multiply(tf.squeeze(self.digitcaps_output), tf.expand_dims(reconstruction_targets, -1)) # [batch_size, 10, 16]


            #Flattening as suggested by the paper
            decoder_input = tf.reshape(digitcaps_output_masked, [BATCH_SIZE, -1]) # [batch_size, 160]


        with tf.variable_scope('Decoder'):
            fc1 = tf.layers.dense(decoder_input, layer1_size, activation=tf.nn.relu, name="FC1") # [batch_size, 512]
            fc2 = tf.layers.dense(fc1, layer2_size, activation=tf.nn.relu, name="FC2") # [batch_size, 1024]
            self.decoder_output = tf.layers.dense(fc2, output_size, activation=tf.nn.sigmoid, name="FC3") # [batch_size, 784]


    def define_loss(self):
        # Margin Loss
        with tf.variable_scope('Margin_Loss'):
            # max(0, m_plus-||v_c||)^2
            positive_error = tf.square(tf.maximum(0., 0.9 - self.v_norm)) # [batch_size, 10, 1, 1]
            # max(0, ||v_c||-m_minus)^2
            negative_error = tf.square(tf.maximum(0., self.v_norm - 0.1)) # [batch_size, 10, 1, 1]
            # reshape: [batch_size, 10, 1, 1] => [batch_size, 10]
            positive_error = tf.reshape(positive_error, shape=(BATCH_SIZE, -1))
            negative_error = tf.reshape(negative_error, shape=(BATCH_SIZE, -1))

            Loss_vec = self.Y * positive_error + 0.5 * (1- self.Y) * negative_error # [batch_size, 10]
            self.margin_loss = tf.reduce_mean(tf.reduce_sum(Loss_vec, axis=1), name="margin_loss")

        # Reconstruction Loss
        with tf.variable_scope('Reconstruction_Loss'):
            ground_truth = tf.reshape(self.X, shape=(BATCH_SIZE, -1))
            self.reconstruction_loss = tf.reduce_mean(tf.square(self.decoder_output - ground_truth))

        # Combined Loss
        with tf.variable_scope('Combined_Loss'):
            self.combined_loss = self.margin_loss + 0.0005 * self.reconstruction_loss

    def define_accuracy(self):
        with tf.variable_scope('Accuracy'):
            correct_predictions = tf.equal(tf.to_int32(tf.argmax(self.Y, axis=1)), self.y_predicted)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

    def define_optimizer(self):
        with tf.variable_scope('Optimizer'):
            optimizer = tf.train.AdamOptimizer()
            self.train_optimizer = optimizer.minimize(self.combined_loss, name="training_optimizer")

    def summary_(self):
        reconstructed_image = tf.reshape(self.decoder_output, shape=(BATCH_SIZE, IMG_WIDTH, IMG_HEIGHT, N_CHANNELS))
        summary_list = [tf.summary.scalar('Loss/margin_loss', self.margin_loss),
                        tf.summary.scalar('Loss/reconstruction_loss', self.reconstruction_loss),
                        tf.summary.image('original', self.X),
                        tf.summary.image('reconstructed', reconstructed_image)]
        self.summary_ = tf.summary.merge(summary_list)
                